In [190]:
import os
import pandas as pd

## My Q's

### (Easy) Q1. Military Branch Consumer Fraud

In [191]:
military_consumer_fraud_df = pd.read_csv("legal/raw/CSVs/2024_CSN_Fraud, Identity Theft, and Other Reports by Military Consumers.csv")

# extract branch fraud table
branch_fraud_df = military_consumer_fraud_df.iloc[7:14]
column_names = list(branch_fraud_df.iloc[0].to_dict().values())
branch_fraud_df.columns = column_names
branch_fraud_df = branch_fraud_df.iloc[1:]

# find branch w/max median fraud loss
max_branch, max_median_fraud_loss = None, None
for idx, branch_row in branch_fraud_df.iterrows():
    median_fraud_loss = int(branch_row['Median Fraud Loss'].replace("$","").replace(",",""))
    if max_branch is None or median_fraud_loss > max_median_fraud_loss:
        max_branch = branch_row['Military Branch']
        max_median_fraud_loss = median_fraud_loss
print(max_branch)

U.S. Space Force


### (Easy) Q2. States with Top Report Category == Identity Theft

In [192]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_State_Top_Ten_Report_Categories.csv")

# extract data
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# group by state, sort by # reports, and add to list of states w/Identity Theft as top category
identity_theft_states = []
for state, state_df in df.groupby('State'):
    state_df['# of Reports'] = state_df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
    state_df.sort_values(['# of Reports'], ascending=False, inplace=True)
    if state_df.iloc[0]['Category'] == 'Identity Theft':
        identity_theft_states.append(state)

print(sorted(identity_theft_states))

['Arizona', 'California', 'Massachusetts', 'Ohio', 'Oklahoma', 'Rhode Island']


### (Easy) Q3. States with "Prizes, Sweepstakes and Lotteries" in their top-10 report categories?

In [193]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_State_Top_Ten_Report_Categories.csv")

# extract data
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# group by state, check if "Prizes, Sweepstakes and Lotteries" is in categories
prizes_states = []
for state, state_df in df.groupby('State'):
    if "Prizes, Sweepstakes and Lotteries" in state_df['Category'].tolist():
        prizes_states.append(state)

print(len(prizes_states))

27


### (Hard) Q1. Gen. Pop. vs. Military

In [194]:
# Get total military reports
military_consumer_reports_df = pd.read_csv("legal/raw/CSVs/2024_CSN_Reports_by_Military_Consumers.csv")
military_consumer_reports_df = military_consumer_reports_df.iloc[1:5]
column_names = list(military_consumer_reports_df.iloc[0].to_dict().values())
military_consumer_reports_df.columns = [col.strip() if isinstance(col, str) else "" for col in column_names]
military_consumer_reports_df = military_consumer_reports_df.iloc[1:]
military_consumer_reports_df['# of Reports'] = military_consumer_reports_df['# of Reports'].apply(lambda elt: int(elt.replace(",","")))
total_military_reports = military_consumer_reports_df['# of Reports'].sum()

# extract military table
military_consumer_reports_df = pd.read_csv("legal/raw/CSVs/2024_CSN_Reports_by_Military_Consumers.csv")
top_military_categories_df = military_consumer_reports_df.iloc[12:23]
column_names = list(top_military_categories_df.iloc[0].to_dict().values())
top_military_categories_df.columns = [col.strip() for col in column_names]
top_military_categories_df = top_military_categories_df.iloc[1:]

# extract report table
report_categories_df = pd.read_csv("legal/raw/CSVs/2024_CSN_Report_Categories.csv", encoding="ISO-8859-1")
column_names = list(report_categories_df.iloc[1].to_dict().values())
report_categories_df.columns = [col.strip() for col in column_names]
report_categories_df = report_categories_df.iloc[2:-4]

# get pct of imposter scams in military
top_military_categories_df['# of Reports'] = top_military_categories_df['# of Reports'].apply(lambda elt: int(elt.replace(",","")))
military_imposter_scams = top_military_categories_df.loc[top_military_categories_df.Category == 'Imposter Scams']['# of Reports'].iloc[0]
pct_military_imposter_scams = military_imposter_scams/total_military_reports

# get pct of imposter scams in general population
pct_general_imposter_scams = report_categories_df.loc[report_categories_df.Category == "Imposter Scams"]['Percentage'].iloc[0]
pct_general_imposter_scams = float(pct_general_imposter_scams.replace("%",""))/100.0

print(pct_military_imposter_scams > pct_general_imposter_scams)

True


### (Hard) Q2. Metro w/Largest Pct. Fraud within State 

In [195]:
import os

# get all filepaths
dir_path = 'legal/raw/CSVs/State MSA Fraud and Other data'
filenames = os.listdir(dir_path)
filepaths = [os.path.join(dir_path, name) for name in filenames]

# compute the metro area with the max pct. of fraud in its own state
five_metro_states = []
max_fraud_metro_area, max_fraud_report_pct = None, 0.0
for idx, filepath in enumerate(filepaths):
    # extract the dataframe
    df = pd.read_csv(filepath)
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # skip states with fewer than 5 metro areas
    if len(df) < 5:
        continue

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
    df['fraud_report_pct'] = df['# of Reports'] / df['# of Reports'].sum()
    df.sort_values(['fraud_report_pct'], ascending=False, inplace=True)

    # get largest fraud metro area for this state and its percent of reports
    state_max_fraud_metro_area = df.iloc[0]['Metropolitan Area']
    state_max_fraud_report_pct = df.iloc[0]['fraud_report_pct']
    if max_fraud_metro_area is None or state_max_fraud_report_pct > max_fraud_report_pct:
        max_fraud_metro_area = state_max_fraud_metro_area
        max_fraud_report_pct = state_max_fraud_report_pct

print(max_fraud_metro_area)

Washington-Arlington-Alexandria, DC-VA-MD-WV Metropolitan Statistical Area


### (Hard) Q3. Metro w/Largest Pct. Fraud within State

In [198]:
import os

# get all filepaths
fraud_dir_path = 'legal/raw/CSVs/State MSA Fraud and Other data'
fraud_filenames = os.listdir(fraud_dir_path)
fraud_filepaths = [os.path.join(fraud_dir_path, name) for name in fraud_filenames]

theft_dir_path = 'legal/raw/CSVs/State MSA Identity Theft Data'
theft_filenames = os.listdir(theft_dir_path)
theft_filepaths = [os.path.join(theft_dir_path, name) for name in theft_filenames]

# compute the number of fraud and identity theft reports in each metro area and return True
# if a metro area has more identity theft reports than fraud reports

# compute mapping from metro area to num fraud reports
metro_area_to_fraud_reports = {}
for idx, filepath in enumerate(fraud_filepaths):
    # extract the dataframe
    df = pd.read_csv(filepath)
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
    metro_area_to_fraud_reports = {
        **pd.Series(df['# of Reports'], index=df['Metropolitan Area'].values).to_dict(),
        **metro_area_to_fraud_reports,
    }

# compute mapping from metro area to num identity theft reports
metro_area_to_identity_reports = {}
for idx, filepath in enumerate(theft_filepaths):
    # extract the dataframe
    df = pd.read_csv(filepath)
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
    metro_area_to_identity_reports = {
        **pd.Series(df['# of Reports'], index=df['Metropolitan Area'].values).to_dict(),
        **metro_area_to_identity_reports,
    }

exists = False
for metro_area, num_identity_reports in metro_area_to_identity_reports.items():
    num_fraud_reports = metro_area_to_fraud_reports.get(metro_area, None)
    if num_fraud_reports is not None and num_identity_reports > num_fraud_reports:
        print(metro_area)
        exists = True
        break

print(exists)

False


### legal-easy-markos-1

In [25]:
qs[0]['query']

'Between 2002 and 2024 inclusive, which year saw the greatest relative increase in identity theft and other reports compared to the previous year?'

In [30]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Number_of_Reports_by_Type.csv")

# extract data frame
df = df.iloc[1:-2]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# convert cols to numeric and compute sum
df['Fraud'] = df['Fraud'].apply(lambda num: int(num.replace(',','')))
df['Identity Theft'] = df['Identity Theft'].apply(lambda num: int(num.replace(',','')))
df['Other'] = df['Other'].apply(lambda num: int(num.replace(',','')))
df['reports'] = df['Fraud'] + df['Identity Theft'] + df['Other']

# compute relative increase in identity theft
df['prev_year_reports'] = df['reports'].shift(1)
df['relative_incr'] = df.apply(lambda row: row['reports'] / row['prev_year_reports'], axis=1)
max_idx = df['relative_incr'].argmax()
print(df.iloc[max_idx]['Year'])

2002


In [26]:
qs[0]['answer']

2002

### legal-easy-markos-2

In [32]:
qs[1]['query']

'In which years were Fraud reports at least 50% of the total reports?'

In [34]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Number_of_Reports_by_Type.csv")

# extract data frame
df = df.iloc[1:-2]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# convert cols to numeric and compute sum
df['Fraud'] = df['Fraud'].apply(lambda num: int(num.replace(',','')))
df['Identity Theft'] = df['Identity Theft'].apply(lambda num: int(num.replace(',','')))
df['Other'] = df['Other'].apply(lambda num: int(num.replace(',','')))
df['reports'] = df['Fraud'] + df['Identity Theft'] + df['Other']

# compute years where fraud is at least 50%
df['fraud_50pct'] = df['Fraud'] >= (0.5 * df['reports'])
print(df[df.fraud_50pct]['Year'].tolist())

['2010', '2011', '2012', '2013', '2014', '2019']


In [35]:
qs[1]['answer']

[2010, 2011, 2012, 2013, 2014, 2019]

### legal-easy-markos-3

In [38]:
qs[2]['query']

"Was there ever a year between 2001-2024 inclusive where 'Other' reports were more than half of the total reports?"

In [39]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Number_of_Reports_by_Type.csv")

# extract data frame
df = df.iloc[1:-2]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# convert cols to numeric and compute sum
df['Fraud'] = df['Fraud'].apply(lambda num: int(num.replace(',','')))
df['Identity Theft'] = df['Identity Theft'].apply(lambda num: int(num.replace(',','')))
df['Other'] = df['Other'].apply(lambda num: int(num.replace(',','')))
df['reports'] = df['Fraud'] + df['Identity Theft'] + df['Other']

# compute years where Other is at least 50%
df['other_half'] = df['Other'] >= (0.5 * df['reports'])
print(df[df.other_half].shape[0] > 0)

False


In [40]:
qs[2]['answer']

'False'

### legal-easy-markos-4

In [42]:
qs[3]['query']

'What is the minimum number of report categories that collectively account for at least 50% of reports in 2024?'

In [47]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Report_Categories.csv", encoding="ISO-8859-1")

# extract data frame
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# convert pct to float
df['Percentage'] = df['Percentage'].apply(lambda pct: float(pct.replace('%','')))

# compute num to reach 50%
num_categories, cum_pct = 0, 0.0
for _, row in df.iterrows():
    cum_pct += row['Percentage']
    num_categories += 1
    if cum_pct >= 50.0:
        break

print(num_categories)

3


In [48]:
qs[3]['answer']

3

### legal-easy-markos-5

In [50]:
qs[4]['query']

'In 2024, what was the ratio of reports between the most frequent and the least frequent category?'

In [51]:
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Report_Categories.csv", encoding="ISO-8859-1")

# extract data frame
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# convert # of reports to int
df['# of Reports'] = df['# of Reports'].apply(lambda pct: float(pct.replace(',','')))

# compute ratio between most and least frequent categories
min_reports = df['# of Reports'].min()
max_reports = df['# of Reports'].max()
print(max_reports / min_reports)

1097.4655312246553


In [52]:
qs[4]['answer']

1097.47

### legal-hard-markos-1

In [54]:
qs[5]['query']

'Which are the top 5 distinct Metropolitan areas in New England in terms of Identity Theft reports?'

In [73]:
new_england_states = ['Connecticut', 'Maine', 'Massachusetts', 'NewHampshire', 'RhodeIsland', 'Vermont']

def extract_state_df(state):
    df = pd.read_csv(f"legal/raw/CSVs/State MSA Identity Theft Data/{state}.csv")

    # extract the dataframe
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))

    return df
    
ne_theft_df = pd.concat([extract_state_df(state) for state in new_england_states])
ne_theft_df.sort_values(by=['# of Reports'], ascending=False, inplace=True)
ne_theft_df.drop_duplicates(subset=['Metropolitan Area'], inplace=True)
ne_theft_df.iloc[:5]['Metropolitan Area'].tolist()

['Boston-Cambridge-Newton, MA-NH Metropolitan Statistical Area',
 'Providence-Warwick, RI-MA Metropolitan Statistical Area',
 'Hartford-West Hartford-East Hartford, CT Metropolitan Statistical Area',
 'Bridgeport-Stamford-Danbury, CT Metropolitan Statistical Area',
 'Worcester, MA Metropolitan Statistical Area']

In [72]:
qs[5]['answer']

['Boston-Cambridge-Newton, MA-NH Metropolitan Statistical Area',
 'Providence-Warwick, RI-MA Metropolitan Statistical Area',
 'Hartford-West Hartford-East Hartford, CT Metropolitan Statistical Area',
 'Bridgeport-Stamford-Danbury, CT Metropolitan Statistical Area',
 'Worcester, MA Metropolitan Statistical Area']

### legal-hard-markos-2

In [75]:
qs[6]['query']

'How many total identity theft reports were there in 2024 from cross-state Metropolitan Statistical Areas?'

In [79]:
import os

def extract_state_df(filename):
    filepath = os.path.join("legal/raw/CSVs/State MSA Identity Theft Data/", filename)
    df = pd.read_csv(filepath)

    # extract the dataframe
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))

    return df

dfs = []
for filename in os.listdir("legal/raw/CSVs/State MSA Identity Theft Data/"):
    dfs.append(extract_state_df(filename))

theft_df = pd.concat(dfs)
theft_df['is_cross_state'] = theft_df['Metropolitan Area'].apply(lambda s: "-" in s.split(',')[-1])
theft_df[theft_df.is_cross_state]['# of Reports'].sum()

593524

In [80]:
qs[6]['answer']

593524

### legal-hard-markos-3

In [81]:
qs[7]['query']

"Among states with at least two metropolitan areas, find the metropolitan area that has the highest fraction of its state's identity theft reports in metropolitan areas. What state does it belong to?"

In [82]:
import os

def extract_state_df(filename):
    filepath = os.path.join("legal/raw/CSVs/State MSA Identity Theft Data/", filename)
    df = pd.read_csv(filepath)

    # extract the dataframe
    df = df.iloc[1:-4]
    column_names = list(df.iloc[0].to_dict().values())
    df.columns = [col.strip() for col in column_names]
    df = df.iloc[1:]

    # compute pct. of total reports for each metro
    df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))

    return df

state_dfs = []
for filename in os.listdir("legal/raw/CSVs/State MSA Identity Theft Data/"):
    state = filename.split('.')[0]
    df = extract_state_df(filename)
    state_dfs.append((state, df))

state_dfs = [(state, df) for state, df in state_dfs if df.shape[0] >= 2]

max_report_frac, max_state = None, None
for state, df in state_dfs:
    total_reports = df['# of Reports'].sum()
    max_report = df['# of Reports'].max()
    report_frac = max_report / total_reports
    if max_report_frac is None or report_frac > max_report_frac:
        max_report_frac = report_frac
        max_state = state

print(max_state)

Delaware


In [83]:
qs[7]['answer']

'Delaware'

### legal-hard-markos-4

In [85]:
qs[8]['query']

'If the 2007 reports were distributed exactly like the 2024 ones, how many reports in 2007 would be Auto Related?'

In [102]:
# get percentage of reports in 2024 that are Auto Related
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Report_Categories.csv", encoding="ISO-8859-1")

# extract data frame
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# convert pct to float
df['Percentage'] = df['Percentage'].apply(lambda pct: float(pct.replace('%','')) / 100)

# compute num to reach 50%
auto_related_pct = df[df.Category == "Auto Related"]['Percentage'].iloc[0]

# get number of reports in 2007
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Report_Count.csv")

# extract data frame
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# compute reports according to distribution
df['# of Reports'] = df['# of Reports'].apply(lambda num: int(num.replace(',','')))
reports_2007 = df[df.Year == "2007"]['# of Reports'].iloc[0]

print(reports_2007 * auto_related_pct)

32541.5888


In [103]:
qs[8]['answer']

32542

### legal-hard-markos-5

In [105]:
qs[9]['query']

'If the 2007 reports were distributed exactly like the 2024 ones, how many identity theft reports in 2007 would be about ages 40 or older?'

In [120]:
# get percentage of reports in 2024 that are Auto Related
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Identity_Theft_Reports_by_Age.csv")

# extract data frame
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# compute total percentage of reports from people ages 40 or older
age_ranges = ["40 - 49", "50 - 59", "60 - 69", "70 - 79", "80 and Over"]
df['# of Reports'] = df['# of Reports'].apply(lambda entry: int(entry.replace(",","")))
forty_or_older_pct = df[df['Age Range'].isin(age_ranges)]['# of Reports'].sum() / df['# of Reports'].sum()

# process reports by type
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Number_of_Reports_by_Type.csv")

# extract data frame
df = df.iloc[1:-2]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]

# compute fraction of 2024 reports that are identity theft related
df['Identity Theft'] = df['Identity Theft'].apply(lambda num: int(num.replace(',','')))
df['Fraud'] = df['Fraud'].apply(lambda num: int(num.replace(',','')))
df['Other'] = df['Other'].apply(lambda num: int(num.replace(',','')))
reports_2024 = df[df.Year == "2024"].iloc[0].to_dict()
total_reports_2024 = reports_2024['Identity Theft'] + reports_2024['Fraud'] + reports_2024['Other']
identity_theft_frac = reports_2024['Identity Theft'] / total_reports_2024

# compute total number of reports in 2007
df = pd.read_csv("legal/raw/CSVs/2024_CSN_Report_Count.csv")
df = df.iloc[1:-4]
column_names = list(df.iloc[0].to_dict().values())
df.columns = [col.strip() for col in column_names]
df = df.iloc[1:]
reports_2007 = int(df[df.Year == "2007"]['# of Reports'].iloc[0].replace(',',''))
print(reports_2007)

# get final number
print(reports_2007 * forty_or_older_pct * identity_theft_frac)

1070447
90934.88670772535


In [108]:
qs[9]['answer']

91263

In [109]:
forty_or_older_pct

0.4860709158093

In [116]:
identity_theft_frac

0.17476953268599488

In [122]:
1070447 * 0.4860709158093 * 0.1754

91262.92714412673